In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
from diskcache import Cache

cache = Cache("match_details.shelve")

@cache.memoize()
def get_page(link):
    print(link)
    return requests.get(link).content

In [3]:
import sys
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [4]:
matches_df = pd.read_csv('dfs/matches/matches.csv', parse_dates=['datetime'])

matches_df

,id,isResult,h_id,h_title,h_short_title,a_id,a_title,a_short_title,goals_h,goals_a,xG_h,xG_a,datetime,forecast_w,forecast_d,forecast_l,season,league
0,14814,False,229,Wolverhampton Wanderers,WOL,89,Manchester United,MUN,NaN,NaN,NaN,NaN,2021-05-22 17:00:00,NaN,NaN,NaN,2020-2021,EPL
1,14813,False,81,West Ham,WHU,74,Southampton,SOU,NaN,NaN,NaN,NaN,2021-05-22 17:00:00,NaN,NaN,NaN,2020-2021,EPL
2,14812,False,238,Sheffield United,SHE,92,Burnley,BUR,NaN,NaN,NaN,NaN,2021-05-22 17:00:00,NaN,NaN,NaN,2020-2021,EPL
3,14811,False,88,Manchester City,MCI,72,Everton,EVE,NaN,NaN,NaN,NaN,2021-05-22 17:00:00,NaN,NaN,NaN,2020-2021,EPL
4,14810,False,87,Liverpool,LIV,78,Crystal Palace,CRY,NaN,NaN,NaN,NaN,2021-05-22 17:00:00,NaN,NaN,NaN,2020-2021,EPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,4751,True,202,Queens Park Rangers,QPR,91,Hull,HUL,0.0,1.0,1.90067,1.117570,2014-08-16 15:00:00,0.5900,0.2449,0.1651,2014-2015,EPL
2656,4750,True,75,Leicester,LEI,72,Everton,EVE,2.0,2.0,1.27830,0.613273,2014-08-16 15:00:00,0.5513,0.2922,0.1565,2014-2015,EPL
2657,4753,True,76,West Bromwich Albion,WBA,77,Sunderland,SUN,2.0,2.0,1.68343,0.991901,2014-08-16 15:00:00,0.5894,0.2676,0.1430,2014-2015,EPL
2658,4754,True,81,West Ham,WHU,82,Tottenham,TOT,0.0,1.0,1.85310,1.017060,2014-08-16 15:00:00,0.5856,0.2648,0.1496,2014-2015,EPL


In [9]:
from tqdm.auto import tqdm

base_url = 'https://understat.com/match/'

dfs = []

for _, row in tqdm(matches_df.iterrows()):
    if row['isResult'] == True:
        match_id = row['id']
        url = base_url + str(match_id)

        #Use requests to get the webpage and BeautifulSoup to parse the page
        res = get_page(url)
        soup = BeautifulSoup(res, 'lxml')
        scripts = soup.find_all('script')

        #get only the shotsData
        strings = scripts[1].string
        # strip unnecessary symbols and get only JSON data 
        ind_start = strings.index("('")+2 
        ind_end = strings.index("')") 
        json_data = strings[ind_start:ind_end] 
        json_data = json_data.encode('utf8').decode('unicode_escape')

        #convert string to json format
        data = json.loads(json_data)
        data_home = data['h']
        data_away = data['a']

        df_h = pd.DataFrame(data_home)
        df_h['season'] = row['season']
        df_h['league'] = row['league']
        df_h['HomeAway'] = 'Home'

        df_a = pd.DataFrame(data_away)
        df_a['season'] = row['season']
        df_a['league'] = row['league']
        df_a['HomeAway'] = 'Away'

        df = pd.concat([df_h, df_a])

        dfs.append(df)

0it [00:00, ?it/s]

https://understat.com/match/14685
https://understat.com/match/14693
https://understat.com/match/14687
https://understat.com/match/14690
https://understat.com/match/14688
https://understat.com/match/14691
https://understat.com/match/14692
https://understat.com/match/14689
https://understat.com/match/14608
https://understat.com/match/14677
https://understat.com/match/14681
https://understat.com/match/14675
https://understat.com/match/14676
https://understat.com/match/14683
https://understat.com/match/14679
https://understat.com/match/14680
https://understat.com/match/14678
https://understat.com/match/14682
https://understat.com/match/14684
https://understat.com/match/14593
https://understat.com/match/14602
https://understat.com/match/14667
https://understat.com/match/14674
https://understat.com/match/14669
https://understat.com/match/14665
https://understat.com/match/14673
https://understat.com/match/14672
https://understat.com/match/14666
https://understat.com/match/14671
https://unders

https://understat.com/match/14102
https://understat.com/match/14101
https://understat.com/match/14100
https://understat.com/match/14099
https://understat.com/match/14098
https://understat.com/match/14097
https://understat.com/match/14096
https://understat.com/match/14095
https://understat.com/match/14094
https://understat.com/match/14093
https://understat.com/match/14092
https://understat.com/match/14091
https://understat.com/match/14090
https://understat.com/match/14087
https://understat.com/match/14086
https://understat.com/match/12022
https://understat.com/match/12013
https://understat.com/match/12021
https://understat.com/match/12020
https://understat.com/match/12019
https://understat.com/match/12018
https://understat.com/match/12017
https://understat.com/match/12016
https://understat.com/match/12015
https://understat.com/match/12014
https://understat.com/match/12006
https://understat.com/match/12007
https://understat.com/match/12003
https://understat.com/match/12011
https://unders

https://understat.com/match/11801
https://understat.com/match/11800
https://understat.com/match/11794
https://understat.com/match/11796
https://understat.com/match/11783
https://understat.com/match/11786
https://understat.com/match/11792
https://understat.com/match/11785
https://understat.com/match/11788
https://understat.com/match/11787
https://understat.com/match/11789
https://understat.com/match/11790
https://understat.com/match/11784
https://understat.com/match/11791
https://understat.com/match/11782
https://understat.com/match/11775
https://understat.com/match/11781
https://understat.com/match/11780
https://understat.com/match/11778
https://understat.com/match/11779
https://understat.com/match/11776
https://understat.com/match/11774
https://understat.com/match/11773
https://understat.com/match/11777
https://understat.com/match/11764
https://understat.com/match/11770
https://understat.com/match/11769
https://understat.com/match/11763
https://understat.com/match/11767
https://unders

https://understat.com/match/9493
https://understat.com/match/9483
https://understat.com/match/9482
https://understat.com/match/9479
https://understat.com/match/9478
https://understat.com/match/9486
https://understat.com/match/9485
https://understat.com/match/9484
https://understat.com/match/9481
https://understat.com/match/9477
https://understat.com/match/9480
https://understat.com/match/9471
https://understat.com/match/9475
https://understat.com/match/9474
https://understat.com/match/9476
https://understat.com/match/9472
https://understat.com/match/9470
https://understat.com/match/9473
https://understat.com/match/9469
https://understat.com/match/9468
https://understat.com/match/9467
https://understat.com/match/9466
https://understat.com/match/9459
https://understat.com/match/9460
https://understat.com/match/9458
https://understat.com/match/9465
https://understat.com/match/9464
https://understat.com/match/9457
https://understat.com/match/9463
https://understat.com/match/9450
https://un

...

https://understat.com/match/3449
https://understat.com/match/3448
https://understat.com/match/3444
https://understat.com/match/3442
https://understat.com/match/3440
https://understat.com/match/3447
https://understat.com/match/3446
https://understat.com/match/3445
https://understat.com/match/3443
https://understat.com/match/3441
https://understat.com/match/3430
https://understat.com/match/3438
https://understat.com/match/3437
https://understat.com/match/3431
https://understat.com/match/3434
https://understat.com/match/3432
https://understat.com/match/3433
https://understat.com/match/3439
https://understat.com/match/3436
https://understat.com/match/3435
https://understat.com/match/3426
https://understat.com/match/3422
https://understat.com/match/3425
https://understat.com/match/3429
https://understat.com/match/3421
https://understat.com/match/3423
https://understat.com/match/3424
https://understat.com/match/3427
https://understat.com/match/3428
https://understat.com/match/3420
https://un

https://understat.com/match/512
https://understat.com/match/511
https://understat.com/match/515
https://understat.com/match/510
https://understat.com/match/508
https://understat.com/match/509
https://understat.com/match/507
https://understat.com/match/506
https://understat.com/match/505
https://understat.com/match/504
https://understat.com/match/503
https://understat.com/match/502
https://understat.com/match/501
https://understat.com/match/500
https://understat.com/match/499
https://understat.com/match/498
https://understat.com/match/494
https://understat.com/match/492
https://understat.com/match/493
https://understat.com/match/497
https://understat.com/match/495
https://understat.com/match/496
https://understat.com/match/491
https://understat.com/match/490
https://understat.com/match/489
https://understat.com/match/488
https://understat.com/match/482
https://understat.com/match/484
https://understat.com/match/483
https://understat.com/match/487
https://understat.com/match/485
https://

https://understat.com/match/256
https://understat.com/match/255
https://understat.com/match/254
https://understat.com/match/253
https://understat.com/match/252
https://understat.com/match/251
https://understat.com/match/250
https://understat.com/match/249
https://understat.com/match/248
https://understat.com/match/247
https://understat.com/match/246
https://understat.com/match/245
https://understat.com/match/244
https://understat.com/match/243
https://understat.com/match/242
https://understat.com/match/241
https://understat.com/match/240
https://understat.com/match/238
https://understat.com/match/239
https://understat.com/match/237
https://understat.com/match/236
https://understat.com/match/235
https://understat.com/match/234
https://understat.com/match/233
https://understat.com/match/232
https://understat.com/match/231
https://understat.com/match/230
https://understat.com/match/229
https://understat.com/match/228
https://understat.com/match/222
https://understat.com/match/225
https://

https://understat.com/match/4445
https://understat.com/match/4440
https://understat.com/match/4512
https://understat.com/match/4511
https://understat.com/match/4510
https://understat.com/match/4509
https://understat.com/match/4508
https://understat.com/match/4507
https://understat.com/match/4506
https://understat.com/match/4505
https://understat.com/match/4504
https://understat.com/match/4503
https://understat.com/match/4502
https://understat.com/match/4501
https://understat.com/match/4500
https://understat.com/match/4499
https://understat.com/match/4498
https://understat.com/match/4497
https://understat.com/match/4496
https://understat.com/match/4495
https://understat.com/match/4494
https://understat.com/match/4493
https://understat.com/match/4492
https://understat.com/match/4491
https://understat.com/match/4487
https://understat.com/match/4485
https://understat.com/match/4486
https://understat.com/match/4489
https://understat.com/match/4488
https://understat.com/match/4490
https://un

https://understat.com/match/4740
https://understat.com/match/4739
https://understat.com/match/4737
https://understat.com/match/4738
https://understat.com/match/4736
https://understat.com/match/4735
https://understat.com/match/4734
https://understat.com/match/4733
https://understat.com/match/4732
https://understat.com/match/4731
https://understat.com/match/4730
https://understat.com/match/4729
https://understat.com/match/4728
https://understat.com/match/4727
https://understat.com/match/4726
https://understat.com/match/4725
https://understat.com/match/4724
https://understat.com/match/4723
https://understat.com/match/4722
https://understat.com/match/4721
https://understat.com/match/4720
https://understat.com/match/4719
https://understat.com/match/4778
https://understat.com/match/4776
https://understat.com/match/4777
https://understat.com/match/4775
https://understat.com/match/4774
https://understat.com/match/4773
https://understat.com/match/4772
https://understat.com/match/4771
https://un

In [10]:
merged_df = pd.concat(dfs)
merged_df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,...,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction,league,HomeAway
0,418051,7,BlockedShot,0.7980000305175782,0.5890000152587891,0.03415979817509651,Anwar El Ghazi,h,5612,OpenPlay,...,14756,Aston Villa,West Bromwich Albion,2,2,2021-04-25 18:00:00,Bertrand Traoré,Pass,EPL,Home
1,418052,8,Goal,0.885,0.5,0.7611688375473022,Anwar El Ghazi,h,5612,Penalty,...,14756,Aston Villa,West Bromwich Albion,2,2,2021-04-25 18:00:00,None,Standard,EPL,Home
2,418056,28,SavedShot,0.7759999847412109,0.649000015258789,0.02307983487844467,Anwar El Ghazi,h,5612,OpenPlay,...,14756,Aston Villa,West Bromwich Albion,2,2,2021-04-25 18:00:00,John McGinn,Pass,EPL,Home
3,418057,30,MissedShots,0.9630000305175781,0.395,0.07580874860286713,Bertrand Traoré,h,695,OpenPlay,...,14756,Aston Villa,West Bromwich Albion,2,2,2021-04-25 18:00:00,Matt Targett,Cross,EPL,Home
4,418058,32,SavedShot,0.8180000305175781,0.7419999694824219,0.024960270151495934,Ross Barkley,h,592,OpenPlay,...,14756,Aston Villa,West Bromwich Albion,2,2,2021-04-25 18:00:00,Matt Targett,Pass,EPL,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,14510,11,SavedShot,0.7619999694824219,0.3160000038146973,0.020646924152970314,Gylfi Sigurdsson,a,714,OpenPlay,...,4749,Manchester United,Swansea,1,2,2014-08-16 12:45:00,Angel Rangel,Pass,EPL,Away
1,14515,27,Goal,0.82,0.5420000076293945,0.06372921168804169,Ki Sung-yueng,a,723,OpenPlay,...,4749,Manchester United,Swansea,1,2,2014-08-16 12:45:00,Gylfi Sigurdsson,Pass,EPL,Away
2,14519,50,SavedShot,0.7180000305175781,0.48900001525878906,0.021781764924526215,Wilfried Bony,a,622,OpenPlay,...,4749,Manchester United,Swansea,1,2,2014-08-16 12:45:00,Wayne Routledge,Pass,EPL,Away
3,14524,71,Goal,0.9219999694824219,0.5259999847412109,0.14339357614517212,Gylfi Sigurdsson,a,714,OpenPlay,...,4749,Manchester United,Swansea,1,2,2014-08-16 12:45:00,Wayne Routledge,Pass,EPL,Away


In [11]:
merged_df.to_csv('dfs/shots/shots.csv', index=False)

In [12]:
del dfs

In [21]:
from tqdm.auto import tqdm

base_url = 'https://understat.com/match/'

dfs = []

for _, row in tqdm(matches_df.iterrows()):
    if row['isResult'] == True:
        match_id = row['id']
        url = base_url + str(match_id)

        #Use requests to get the webpage and BeautifulSoup to parse the page
        res = get_page(url)
        soup = BeautifulSoup(res, 'lxml')
        scripts = soup.find_all('script')

        #get only the shotsData
        strings = scripts[2].string
        # strip unnecessary symbols and get only JSON data 
        ind_start = strings.index("('")+2 
        ind_end = strings.index("')") 
        json_data = strings[ind_start:ind_end] 
        json_data = json_data.encode('utf8').decode('unicode_escape')

        #convert string to json format
        data = json.loads(json_data)
        data_home = data['h']
        data_away = data['a']
        
        data_home = [data_home[key] for key in data_home]
        data_away = [data_away[key] for key in data_away]

        df_h = pd.DataFrame(data_home)
        df_h['season'] = row['season']
        df_h['league'] = row['league']
        df_h['match_id'] = row['id']
        df_h['HomeAway'] = 'Home'

        df_a = pd.DataFrame(data_away)
        df_a['season'] = row['season']
        df_a['league'] = row['league']
        df_a['match_id'] = row['id']
        df_a['HomeAway'] = 'Away'

        df = pd.concat([df_h, df_a])

        dfs.append(df)

0it [00:00, ?it/s]

In [22]:
roster_df = pd.concat(dfs)
roster_df

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,key_passes,assists,xA,xGChain,xGBuildup,positionOrder,season,league,match_id,HomeAway
0,466340,0,0,0,0,90,4401,71,GK,Emiliano Martinez,...,0,0,0,0.19965820014476776,0.19965820014476776,1,2020-2021,EPL,14756,Home
1,466341,0,0,2,0.0507240854203701,90,1685,71,DR,Ahmed Elmohamady,...,3,0,0.06234762817621231,0.3797950744628906,0.30373573303222656,2,2020-2021,EPL,14756,Home
2,466342,0,0,3,0.10185212641954422,90,7726,71,DC,Ezri Konsa Ngoyo,...,0,0,0,0.03415979817509651,0.03415979817509651,3,2020-2021,EPL,14756,Home
3,466343,0,1,1,0.08413901180028915,90,1024,71,DC,Tyrone Mings,...,1,0,0.13328632712364197,0.40749111771583557,0.40749111771583557,3,2020-2021,EPL,14756,Home
4,466344,0,0,0,0,90,884,71,DL,Matt Targett,...,4,0,0.1633511483669281,0.29705607891082764,0.19628706574440002,4,2020-2021,EPL,14756,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,17156,0,0,0,0,90,719,84,AML,Wayne Routledge,...,2,1,0.16517533361911774,0.22890454530715942,0.06372921168804169,13,2014-2015,EPL,4749,Away
10,17157,0,0,1,0.021781764924526215,80,622,84,FW,Wilfried Bony,...,0,0,0,0.22890454530715942,0.2071227878332138,15,2014-2015,EPL,4749,Away
11,17160,0,0,0,0,34,4448,84,Sub,Dwight Tiendalli,...,0,0,0,0,0,17,2014-2015,EPL,4749,Away
12,17158,0,0,0,0,20,715,84,Sub,Jefferson Montero,...,0,0,0,0.14339357614517212,0.14339357614517212,17,2014-2015,EPL,4749,Away


In [25]:
roster_df.to_csv('dfs/rosters/rosters.csv', index=False)